```Copyright (c) 2019, RTE (http://www.rte-france.com)
This Source Code Form is subject to the terms of the Mozilla Public
License, v. 2.0. If a copy of the MPL was not distributed with this
file, You can obtain one at http://mozilla.org/MPL/2.0/.```

# Pypowsybl demo: powsybl code in a jupiter notebook

## Activates powsybl java-python bridge

In [ ]:
import pypowsybl_core

import jpype
import jpype.imports
from jpype.types import*

def init_jvm(jvmpath=None):
    if jpype.isJVMStarted():
        return
    jpype.startJVM(jpype.getDefaultJVMPath(), "-ea", convertStrings=False)

init_jvm()

# imports powsybl classes (java)
from java.nio.file import Path, Paths
from com.powsybl.loadflow import LoadFlow, LoadFlowResult, LoadFlowParameters
from com.powsybl.iidm.export import Exporters
from java.io import File
# needed to do this way, because of the underscore character, which has a special meaning in Python
pimport = jpype.JPackage('com.powsybl.iidm.import_')

from com.powsybl.substationdiagram import SubstationDiagram,VoltageLevelDiagram
from com.powsybl.substationdiagram.layout import HorizontalSubstationLayoutFactory,LayoutParameters,SubstationLayoutFactory
from com.powsybl.substationdiagram.layout import VoltageLevelLayoutFactory,PositionVoltageLevelLayoutFactory,PositionFree
from com.powsybl.substationdiagram.library import ComponentLibrary,ResourcesComponentLibrary

from IPython.core.display import display,SVG,HTML

## Defines some utility functions

In [ ]:
#print network's flow to stdout
def dumpLinesFlow(network, headline):
    print("\n{}".format(headline))
    lines = network.getLines()
    print("line id;terminal1.I;terminal2.I")
    for line in lines:
        print("{};{};{}".format(line.getId(), line.getTerminal1().getI(), line.getTerminal2().getI()))

#create an svg file, for a specific substation
def writeSubstationSvg(substation, outfile):
    parameters = LayoutParameters()
    voltageLevelLayoutFactory = PositionVoltageLevelLayoutFactory(PositionFree())
    substationLayoutFactory = HorizontalSubstationLayoutFactory()
    componentLibrary = ResourcesComponentLibrary("/ConvergenceLibrary")
    SubstationDiagram.build(substation, substationLayoutFactory, voltageLevelLayoutFactory, True).writeSvg(componentLibrary, parameters, 
        substation.getNetwork(), Paths.get(outfile))

#display an svg file, setting/overwriting its declared size: width, height
def showSvg(svgs, width=700, height=700):
    SVG_HEAD = '''<?xml version="1.0" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">'''
    SVG_START = '''<svg width="{w:}px" height="{h:}px" version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink= "http://www.w3.org/1999/xlink">'''
    SVG_END = '</svg>'
    return display(SVG(SVG_HEAD + SVG_START.format(w=width, h=height) + SVG(svgs)._repr_svg_() + SVG_END))

#display a substation, inline 
def displaySubstation(substation):
    import uuid
    svgFileName = str("/tmp/substation_{}.svg".format(uuid.uuid4()))
    writeSubstationSvg(substation,svgFileName)
    showSvg(svgFileName)



## Loads a test network from an xiidm file**

In [ ]:
network = pimport.Importers.loadNetwork(Paths.get("./eurostag-tutorial-example1.xml"))

## Dumps the network's lines flow, before running a loadflow

In [ ]:
dumpLinesFlow(network,"Flow on lines for network: {} - PRE-LOADFLOW".format(network.getId()))

## Displays substation P1's graph inline

In [ ]:
displaySubstation(network.getSubstation("P1"))

## Runs a loadflow

In [ ]:
result = LoadFlow.run(network, LoadFlowParameters.load())
print("Loadflow result = isOK: {}, metrics: {}\n".format(result.isOk(), result.getMetrics()))

## Dumps network's lines flow, after the loadflow

In [ ]:
dumpLinesFlow(network,"Flow on lines for network: {} - POST-LOADFLOW".format(network.getId()))

## Displays substation P1's graph inline, after the loadflow

In [ ]:
displaySubstation(network.getSubstation("P1"))

## Dumps generator GEN's P and Q ,at its terminal

In [ ]:
print("P={}".format(network.getGenerator("GEN").getTerminal().getP()))
print("Q={}".format(network.getGenerator("GEN").getTerminal().getQ()))